# Project: Sharktale, SharkTrack
## Data Cleaning

In [ ]:
!pip install xlrd #!pip install country_converter, keep this here pls

### Import of Libraries

In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

### Loading the Excel Data

In [ ]:
df = pd.read_excel('https://www.sharkattackfile.net/spreadsheets/GSAF5.xls')
df

### Data at a Glance 
Reviewing how the database is organised by using .head(), .info(), .describe(), .tail(), .columns. This will help me on how to pepare the data cleaning process.

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.columns

### Columns Cleaning and Renaming 

I have reviewed the columns and decided to do the following: 
- Remove extra " " and also replacing extra " " with "_". 
- Dropping the following columns: "Case_Number_1", "Unnamed_21". "Unnamed_22" . 
- Renaming:  'Fatal Y/N': 'Fatal', 'Sex': 'Gender',  'pdf': 'PDF',  and  'Species ': 'Species', 'href': 'Link','original order': 'Original_Order', 

In [ ]:
df = df.rename(columns= {
    'Fatal Y/N': 'Fatal',
    'Species ': 'Species',
    'pdf': 'PDF', 
    'Sex': 'Gender',
    'href formula': 'Href_formula', 
    'href': 'Link',
    'Case Number': 'Case_Number', 
    'Case Number.1': 'Case_Number_1',
    'original order': 'Original_Order', 
    'Unnamed: 21': 'Unnamed_21', 
    'Unnamed: 22': 'Unnamed_22'
})
df.columns

In [ ]:
df["Case_Number"].isin(df["Case_Number_1"]).value_counts() # There is less than 10% difference 

In [ ]:
# Dropping Columns
df = df.drop(columns=["Unnamed_21", "Unnamed_22", "PDF", "Href_formula", "Case_Number_1"])
df.columns

### Cleaning Column 'Month'

In [ ]:
def finding_month(date): 
    months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Oct", "Nov", "Dec"]
    #If it founds the info of a given month, it returns an output of this cleaning function
    try:
        for month in months: 
            if re.search(month, date):
                return month
    except TypeError:
        pass 
  
df["Month"] = df["Date"]
df["Month"] = df["Date"].apply(finding_month)
df["Month"].fillna(value="Unknown", inplace=True)
print(df["Month"].value_counts())  # Check unique values

### Data Cleaning for the Column 'Type' 

In [ ]:
df.Type.unique() 

In [ ]:
print(df["Type"].value_counts()) 

In [ ]:
print(df["Type"].value_counts(dropna=False)) # I wanted to review the Nan 

In [ ]:
#Reviewing how many were provoked and unprovoked 
print(df["Type"].unique())

In [ ]:
df["Type"] = df["Type"].str.strip()
df["Type"] = df["Type"].replace({
    " Provoked": "Provoked", 
    "Unconfirmed": "Unknown",
    "?" : "Unknown",
    "Invalid": "Unknown",
    "Under investigation": "Unknown",
    "Questionable": "Unknown", 
    "Watercraft" : "Water Vehicle", 
    "Boat" : "Water Vehicle"
})

df["Type"] = df["Type"].fillna("Unknown")
print(df["Type"].unique())

## Data Cleaning for the Column 'Gender'

In [ ]:
df["Gender"] = df["Gender"].replace({
    " M": "M", 
    "M ": "M", 
    "M x 2": "M", 
    " nan": "Unknown", 
    "lli": "Unknown", 
    "N": "M", 
    ".": "Unknown"
    })
df["Gender"] = df["Gender"].fillna("Unknown")
df.Gender.unique()

### Data Cleaning for the Column 'Species'